# Score review via services


## Obtain Lablled Reviews
In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [ ]:
from nltk import download

download('movie_reviews')

## Load the Data
The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [ ]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)
    
reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)
    
print(len(reviews_neg))
print(len(reviews_neg))

In [ ]:
import boto3
import json
import pandas as pd

access_key_id=''
secret_access_key=''
comprehend = boto3.client(service_name='comprehend',region_name='eu-west-1',aws_access_key_id=access_key_id,aws_secret_access_key=secret_access_key )


def score_review_aws(review):
    print(review)
    review =(review[:500])
    print(review)
    result = comprehend.detect_sentiment(Text=review,LanguageCode='en')
    rawSentiment = result["Sentiment"]
   # print(rawSentiment)
    result_mapping = {'POSITIVE':'pos', 'NEGATIVE':'neg', 'NEUTRAL':'pos','MIXED':'neg'}
    sentiment = result_mapping[rawSentiment]
   # print(sentiment)
    print('.', end ='')
    
    return sentiment

In [ ]:
score_review_aws("Really happy to see you joining this diner!")

## Score each Review

Now, we can use the function you defined to score each of the reviews

In [ ]:
max_review = 10
reviews_pos2 = (reviews_pos[:max_review])
reviews_neg2 = (reviews_neg[:max_review])

results_pos = []
for review in reviews_pos2:
    result = score_review_aws(review)
    results_pos.append(result)

print("Positive Done")
    
results_neg = []
for review in reviews_neg2:
    result = score_review_aws(review)
    results_neg.append(result)
    
print("Negative Done")

## Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [ ]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))